In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scipy.stats as stats
# import tensorflow_probability as tfp
import gpytorch
from gpytorch.distributions import MultivariateNormal
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import gamma, norm, beta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder,  MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import warnings

ModuleNotFoundError: No module named 'gpytorch'

In [ ]:
#!pip install gpytorch

In [ ]:
# !pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

## 1. Data preprocess

In [ ]:
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
data = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets.values - 1 ## 1 subtracted for (0 = Good,  1 = Bad) labelling
  
# metadata 
#print(statlog_german_credit_data.metadata) 
  
# variable information 
#print(statlog_german_credit_data.variables) 

In [ ]:
df=data[statlog_german_credit_data.variables.name[:-1]]
df_full=df.copy()
df_full.columns=statlog_german_credit_data.variables.description[:-1].to_list()

In [ ]:
# Define preprocessing steps
numeric_features = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11', 'Attribute13', 'Attribute16', 'Attribute18']
binary_features = ['Attribute19', 'Attribute20']
categorical_features = ['Attribute1', 'Attribute3', 'Attribute4', 'Attribute6', 'Attribute7', 'Attribute9', 'Attribute10', 'Attribute12', 'Attribute14', 'Attribute15', 'Attribute17']

# Apply LabelEncoder to binary features
label_encoders = {}
for feature in binary_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le  # Store the encoder for future use (e.g., inverse transform)

# Pipeline for numeric features: Imputation and Min-Max Scaling between -1 and 1
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler(feature_range=(-1, 1)))
])

# Pipeline for categorical features: Imputation, OneHotEncoding, and Min-Max Scaling between -1 and 1
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False)),  # Set sparse=False for easy concatenation
    ('scaler', MinMaxScaler(feature_range=(-1, 1)))  # Scaling the one-hot encoded features
])

# For binary features, use Min-Max Scaling as well
binary_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler(feature_range=(-1, 1)))
])

# Combine the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_features)
    ]
)

# Apply the preprocessing steps to the DataFrame
df2 = preprocessor.fit_transform(df)

# If you want to convert it back to a DataFrame for ease of use
# Create column names for the one-hot encoded features
onehot_feature_names = list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out())

# Combine all feature names
all_feature_names = numeric_features + onehot_feature_names + binary_features

# Create the processed DataFrame
df2 = pd.DataFrame(df2, columns=all_feature_names)

# Show the processed DataFrame
df2

/Users/rellaxiao/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Attribute2,Attribute5,Attribute8,Attribute11,Attribute13,Attribute16,Attribute18,x0_A11,x0_A12,x0_A13,...,x8_A143,x9_A151,x9_A152,x9_A153,x10_A171,x10_A172,x10_A173,x10_A174,Attribute19,Attribute20
0,-0.941176,-0.898867,1.000000,1.000000,0.714286,-0.333333,-1.0,1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0
1,0.294118,-0.372620,-0.333333,-0.333333,-0.892857,-1.000000,-1.0,-1.0,1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
2,-0.764706,-0.796853,-0.333333,0.333333,0.071429,-1.000000,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
3,0.117647,-0.160119,-0.333333,1.000000,-0.071429,-1.000000,1.0,1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
4,-0.411765,-0.491581,0.333333,1.000000,0.214286,-0.333333,1.0,1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.764706,-0.836470,0.333333,1.000000,-0.571429,-1.000000,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
996,-0.235294,-0.603059,1.000000,1.000000,-0.250000,-1.000000,-1.0,1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
997,-0.764706,-0.939034,1.000000,1.000000,-0.321429,-1.000000,-1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0
998,0.205882,-0.824475,1.000000,1.000000,-0.857143,-1.000000,-1.0,1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0


## 2. Define reward functions

In [ ]:
# Suppress warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Call the seed setting function
set_seed(42)

In [ ]:
# RewardGenerator and generate_data functions (from your code)
class RewardGenerator:
    def __init__(self, drift_rate=0.005, seasonal_period=400):
        self.drift_rate = drift_rate
        self.seasonal_period = seasonal_period
        self.t = 0  # Initialize time step
    
    def generate_rewards1(self, context, action, optimal_action, n = 3):
        '''
        Generates rewards using a simple linear function with noise.

        Parameters:
        - context: np.array, the feature context
        - action: int, the action taken
        - optimal_action: int, the optimal action for this context

        Returns:
        - reward: float, the generated reward
        '''

        # Sample `n` context features
        if len(context) >= n:
            sampled_context = np.random.choice(context, size=n, replace=False)
        else:
            sampled_context = context  # Use all features if fewer than n
        
        # Linear reward based on context sum
        context_factor = np.sum(sampled_context)
        
        # Add a base reward depending on whether the action is optimal
        base_reward = 1.5 if action == optimal_action else 0.8

        # Linear reward calculation with noise
        reward = base_reward + 0.1 * context_factor + np.random.normal(0, 0.1)

        return reward

    def generate_rewards2(self, context, action, optimal_action, n=3):
        '''
        Generates rewards with added complexity from context sampling and non-stationarity.

        Parameters:
        - context: np.array, the feature context
        - action: int, the action taken
        - optimal_action: int, the optimal action for this context
        - n: int, number of context features to sample for reward generation

        Returns:
        - reward: float, the generated reward
        '''
        # Sample `n` context features
        if len(context) >= n:
            sampled_context = np.random.choice(context, size=n, replace=False)
        else:
            sampled_context = context  # Use all features if fewer than n
        
        # Compute a context factor based on the sampled features
        context_factor = np.sum(sampled_context)
        
        # Non-stationarity (drift and seasonality)
        time_drift = self.drift_rate * self.t
        seasonality = np.sin(2 * np.pi * self.t / self.seasonal_period)

        # Calculate reward based on action and non-stationary factors
        if action == optimal_action:
            mean = 1 + 0.2 * context_factor + time_drift + seasonality
            std = 0.1  # Small variance for optimal action
        else:
            mean = 0 + 0.5 * context_factor + time_drift + seasonality
            std = 0.4  # Higher variance for non-optimal action
        
        # Sample reward from Gaussian distribution
        reward = np.random.normal(mean, std)

        # Increment time step for next call
        self.t += 1
        
        return reward
    

    def generate_rewards3(self, context, action, optimal_action, n=3):
        '''
        Generates rewards using a piecewise linear function with context-dependent intervals.
        '''
        # Sample `n` context features
        if len(context) >= n:
            sampled_context = np.random.choice(context, size=n, replace=False)
        else:
            sampled_context = context  # Use all features if fewer than n

        # Compute a context factor based on the sampled features
        context_factor = np.sum(sampled_context)
        
        # Piecewise modification based on context factor
        if context_factor < 0.5:
            context_factor = 1
        elif 0.5 <= context_factor < 0.6:
            context_factor = -10 * context_factor + 4
        else:
            context_factor = 0
        
        # Non-stationarity (drift and seasonality)
        time_drift = self.drift_rate * self.t
        seasonality = np.sin(2 * np.pi * self.t / self.seasonal_period)

        # Calculate reward based on action and non-stationary factors
        if action == optimal_action:
            mean = 1 + 0.3 * context_factor + time_drift + seasonality
            std = 0.1  # Small variance for optimal action
        else:
            mean = 0 + 0.6 * context_factor + time_drift + seasonality
            std = 0.3  # Higher variance for non-optimal action
        
        # Sample reward from Gaussian distribution
        reward = np.random.normal(mean, std)

        # Increment time step for next call
        self.t += 1
        
        return reward

    def generate_rewards4(self, context, action, optimal_action, n=3):
        '''
        Generates rewards with a weighted linear combination of context features, drift, and seasonality.
        '''
        # Sample `n` context features
        if len(context) >= n:
            sampled_context = np.random.choice(context, size=n, replace=False)
        else:
            sampled_context = context  # Use all features if fewer than n
        
        # Assign weights to context features
        context_weights = np.array([0.3, 0.5, 0.2])[:len(sampled_context)]
        # Weighted sum of context features
        context_factor = np.dot(sampled_context, context_weights)  

        # Non-stationary components: drift and seasonality
        time_drift = self.drift_rate * self.t  
        seasonality = 1.5 * np.sin(2 * np.pi * self.t / (self.seasonal_period * 4))

        # Calculate reward based on action and non-stationary factors
        if action == optimal_action:
            mean = 1.2 + 0.4 * context_factor + time_drift + seasonality
            std = 0.1  # Small variance for optimal action
        else:
            mean = 0.5 + 0.7 * context_factor + time_drift + seasonality
            std = 0.3  # Higher variance for non-optimal action

        # Sample reward from Gaussian distribution
        reward = np.random.normal(mean, std)

        # Increment time step for next call
        self.t += 1
        
        return reward

    def generate_rewards5(self, context, action, optimal_action, n=3):
        '''
        Reward function that models heteroscedastic rewards where reward variance depends on the context.
        '''
        # Randomly generate theta_heteroscedastic and theta_var
        theta_heteroscedastic = np.random.randn(n)  # Random vector for mean reward
        theta_var = np.random.randn(n)              # Random vector for variance

        # Sample `n` context features
        if len(context) >= n:
            sampled_context = np.random.choice(context, size=n, replace=False)
        else:
            sampled_context = context  # Use all features if fewer than n
    
        # Linear reward based on context sum
        context_factor = np.sum(sampled_context)
    
        # Calculate the mean reward (dot product with theta_heteroscedastic)
        base_reward = np.dot(sampled_context, theta_heteroscedastic)
    
        # Calculate the context-dependent variance (dot product with theta_var)
        context_variance = np.abs(np.dot(sampled_context, theta_var)) 
    
        # Add a base reward depending on whether the action is optimal
        optimal_bonus = 1.5 if action == optimal_action else 0.8
    
        # Linear reward calculation with noise
        reward = optimal_bonus + base_reward + 0.1 * context_factor + np.random.normal(0, context_variance)
        # reward = optimal_bonus + base_reward + np.random.normal(0, context_variance)
        
        return reward



In [ ]:
# Example usage in your data generation function
def generate_data(context, total_samples, n_actions, optimal_actions, reward_type):
    rewards = torch.zeros(total_samples, n_actions)
    
    reward_gen = RewardGenerator()

    for i in range(total_samples):
        for action in range(n_actions):
            # Generate rewards based on the given logic
            if reward_type == 'lin1':
                rewards[i, action] = reward_gen.generate_rewards1(context[i], action,  optimal_actions[i])
            elif reward_type == 'lin2':
                rewards[i, action] = reward_gen.generate_rewards2(context[i], action,  optimal_actions[i])
            elif reward_type == 'lin3':
                rewards[i, action] = reward_gen.generate_rewards3(context[i], action,  optimal_actions[i])
            elif reward_type == 'lin4':
                rewards[i, action] = reward_gen.generate_rewards4(context[i], action,  optimal_actions[i])
            elif reward_type == 'lin5':
                rewards[i, action] = reward_gen.generate_rewards5(context[i], action,  optimal_actions[i])   

    return rewards


# Function to generate actions from a normal distribution and clip them to valid action range
# def generate_actions(total_samples):
#    return  np.random.choice([0, 1], size=total_samples)

def generate_actions(total_samples, bias=0.6):
    # Generate actions with the specified bias
    actions = np.random.choice([0, 1], size=total_samples, p=[1 - bias, bias])
    return actions

# Define the data generation and splitting function
def generate_data_and_split(X, total_samples, n_actions, optimal_actions, reward_type):
    
    rewards = generate_data(X, total_samples, n_actions, optimal_actions, reward_type)
    split_index = int(total_samples * 0.9)
    X_train = X[:split_index]
    historical_actions = generate_actions(split_index)
    historical_rewards = rewards[:split_index]
    
    X_val = X[split_index:]
    optimal_actions_val = optimal_actions[split_index:]
    rewards_val = rewards[split_index:]
    
    return X_train, X_val, historical_actions, historical_rewards, optimal_actions_val, rewards_val

## 3.1 LinUCB with DB

In [ ]:
class LinUCB_DR:
    def __init__(self, n_actions, context_dim, alpha=0.5):
        self.n_actions = n_actions
        self.context_dim = context_dim
        self.alpha = alpha
        self.A = [np.eye(context_dim) for _ in range(n_actions)]  # Identity matrices for each action
        self.b = [np.zeros(context_dim) for _ in range(n_actions)]  # Zero vectors for each action
        self.propensities = None

    def calculate_propensity_scores(self, historical_actions):
        # Calculate propensity scores based on historical action frequencies
        total_samples = len(historical_actions)
        action_freq = np.bincount(historical_actions, minlength=self.n_actions) / total_samples
        self.propensities = action_freq[historical_actions]
        return self.propensities

    def update(self, action, reward, context):
        # Update A and b for the taken action
        self.A[action] += np.outer(context, context)
        self.b[action] += reward * context

    def predict(self, X):
        # Predict the reward for each action in the context X
        p = np.zeros((X.shape[0], self.n_actions))
        for a in range(self.n_actions):
            theta_a = np.linalg.solve(self.A[a], self.b[a])  # Solve A * theta = b
            p[:, a] = X @ theta_a + self.alpha * np.sqrt(np.sum(X @ np.linalg.inv(self.A[a]) * X, axis=1))
        return p

    def doubly_robust_estimator(self, X, chosen_actions, historical_rewards, historical_actions, all_true_rewards):
        # Calculate doubly robust reward estimates
        N = len(chosen_actions)
        dr_rewards = np.zeros(N)
        propensities = self.propensities if self.propensities is not None else self.calculate_propensity_scores(historical_actions)

        numerator_all = np.exp(self.predict(X))
        for i in range(N):
            Ai = chosen_actions[i]
            Ci = X[i]
            Ri = historical_rewards[i, historical_actions[i]]
            # pi_ratio = propensities[i] / propensities[chosen_actions[i]]
            # pi_ratio = propensities[chosen_actions[i]] / propensities[historical_actions[i]]
            
            # numerator = p/sum(p)  (softmax)
            # numerator = numerator_all[i] / np.sum(numerator_all[i])
            pi_ratio = numerator_all[i, Ai] / np.sum(numerator_all[i])
            # print(pi_ratio)

            predicted_reward = all_true_rewards[i, Ai]
            
            # dr_rewards[i] = Ri * pi_ratio + 1 / pi_ratio * predicted_reward
            dr_rewards[i] = pi_ratio * (Ri - predicted_reward) + predicted_reward

        return dr_rewards


## 3.2 GPThompson

In [ ]:
# Define the GP model
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, lengthscale=1.0, outputscale=1.0):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, lengthscale=lengthscale)
        )
        self.covar_module.outputscale = outputscale

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
        

# Define the GP Thompson Sampling class
class GPThompsonSampling:
    def __init__(self, n_actions, context_dim, num_samples=10, lengthscale=1.0, outputscale=1.0):
        self.n_actions = n_actions
        self.context_dim = context_dim
        self.num_samples = num_samples
        self.lengthscale = lengthscale
        self.outputscale = outputscale
        self.models = [None] * n_actions
        self.likelihoods = [gpytorch.likelihoods.GaussianLikelihood() for _ in range(n_actions)]
        # use dr
        self.propensities = None
    
    def update(self, actions, rewards, contexts):
        contexts = torch.tensor(contexts, dtype=torch.float32)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.long)

        for action in range(self.n_actions):
            indices = (actions == action)
            if indices.sum().item() == 0:
                continue

            X_train_action = contexts[indices]
            y_train_action = rewards[indices, action]

            # Initialize and train the GP model for this action
            self.models[action] = ExactGPModel(X_train_action, y_train_action, self.likelihoods[action], 
                                               lengthscale=self.lengthscale, outputscale=self.outputscale)
            self.models[action].train()
            optimizer = torch.optim.Adam(self.models[action].parameters(), lr=0.1)
            mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihoods[action], self.models[action])

            for _ in range(100):  # Number of training epochs
                optimizer.zero_grad()
                output = self.models[action](X_train_action)
                
                if y_train_action.dim() > 1:
                    y_train_action = y_train_action.squeeze(-1)

                loss = -mll(output, y_train_action)
                
                if loss.numel() != 1:
                    raise ValueError(f"Expected scalar loss, but got shape {loss.shape}")

                loss.backward()
                optimizer.step()
                
    def predict(self, context):
        context = torch.tensor(context, dtype=torch.float32)
        means = np.zeros((context.shape[0], self.n_actions))
        stddevs = np.zeros((context.shape[0], self.n_actions))
        for action in range(self.n_actions):
            if self.models[action] is None:
                means[:, action] = np.random.randn(context.shape[0])
                stddevs[:, action] = np.ones(context.shape[0])
            else:
                self.models[action].eval()
                gp_samples = self.sample_from_gp_posterior(self.models[action], context, self.num_samples)
                means[:, action] = gp_samples.mean(dim=0).numpy()
                stddevs[:, action] = gp_samples.std(dim=0).numpy()
        return means, stddevs

    def sample_from_gp_posterior(self, model, context, num_samples):
        with torch.no_grad():
            posterior = model(context)
            samples = posterior.sample(sample_shape=torch.Size([num_samples]))
        return samples

    def calculate_propensity_scores(self, historical_actions):
        # Calculate propensity scores based on historical action frequencies
        total_samples = len(historical_actions)
        action_freq = np.bincount(historical_actions, minlength=self.n_actions) / total_samples
        self.propensities = action_freq[historical_actions]
        return self.propensities
    
    def doubly_robust_estimator(self, X, chosen_actions, historical_rewards, historical_actions, all_true_rewards):
        # Calculate doubly robust reward estimates
        N = len(chosen_actions)
        dr_rewards = np.zeros(N)
        propensities = self.propensities if self.propensities is not None else self.calculate_propensity_scores(historical_actions)

        numerator_all = np.exp(self.predict(X)[0])
        for i in range(N):
            Ai = chosen_actions[i]
            Ci = X[i]
            Ri = historical_rewards[i, historical_actions[i]]
            pi_ratio = numerator_all[i, Ai] / np.sum(numerator_all[i])
            predicted_reward = all_true_rewards[i, Ai]
            
            dr_rewards[i] = pi_ratio * (Ri - predicted_reward) + predicted_reward
        return dr_rewards


## 4. Backtest

In [ ]:
def run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim, num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=20, reward_type='lin1'):
    # Generate and split the dataset
    X_train, X_val, historical_actions, historical_rewards, optimal_actions_val, rewards_val = generate_data_and_split(
        X, total_samples, n_actions, optimal_actions, reward_type)

    # Convert PyTorch tensors to NumPy arrays if needed
    if isinstance(X_train, torch.Tensor):
        X_train = X_train.detach().numpy()
    if isinstance(X_val, torch.Tensor):
        X_val = X_val.detach().numpy()
    if isinstance(historical_rewards, torch.Tensor):
        historical_rewards = historical_rewards.detach().numpy()
    if isinstance(historical_actions, torch.Tensor):
        historical_actions = historical_actions.detach().numpy()
     
    # GP Thompson Sampling
    gp_ts_model = GPThompsonSampling(n_actions, context_dim, num_samples, lengthscale, outputscale)
    gp_ts_model.update(historical_actions, historical_rewards, X_train)
    gp_ts_means, gp_ts_stddevs = gp_ts_model.predict(X_val)
    gp_ts_preds = np.argmax(gp_ts_means, axis=1)
    
    
    # LinUCB_DR model
    lin_UCB_DR_model = LinUCB_DR(n_actions, context_dim, alpha=0.5)
    lin_UCB_DR_model.calculate_propensity_scores(historical_actions)  # Calculate propensities internally
    
    # Update the model with historical data
    for i in range(len(historical_actions)):
        lin_UCB_DR_model.update(historical_actions[i], historical_rewards[i, historical_actions[i]], X_train[i])
    
    # Generate predictions on validation data
    lin_UCB_preds = lin_UCB_DR_model.predict(X_val)
    lin_UCB_preds = np.argmax(lin_UCB_preds, axis=1)

    
    # Generate true rewards based on optimal actions
    all_true_rewards = np.zeros((X_val.shape[0], n_actions))
    
    reward_gen = RewardGenerator()
    
    for action in range(n_actions):
        for i, opt_action in enumerate(optimal_actions_val):
            if reward_type == 'lin1':
                all_true_rewards[i, action] = reward_gen.generate_rewards1(X_val[i], action, opt_action)
            elif reward_type == 'lin2':
                all_true_rewards[i, action] = reward_gen.generate_rewards2(X_val[i], action, opt_action)
            elif reward_type == 'lin3':
                all_true_rewards[i, action] = reward_gen.generate_rewards3(X_val[i], action, opt_action)
            elif reward_type == 'lin4':
                all_true_rewards[i, action] = reward_gen.generate_rewards4(X_val[i], action, opt_action)
            elif reward_type == 'lin5':
                all_true_rewards[i, action] = reward_gen.generate_rewards5(X_val[i], action, opt_action)

    # true_rewards = np.max(all_true_rewards, axis=1)
    
    # Calculate doubly robust rewards
    linucb_rewards = lin_UCB_DR_model.doubly_robust_estimator(X_val, lin_UCB_preds, historical_rewards, historical_actions, all_true_rewards)
    gp_ts_rewards = gp_ts_model.doubly_robust_estimator(X_val, gp_ts_preds, historical_rewards, historical_actions, all_true_rewards)

    # return average rewards
    print(f"Average Rewards LinUCB: {np.mean(linucb_rewards):.4f}, LinUCB Accuracy: {accuracy_score(optimal_actions_val, lin_UCB_preds):.4f}")
    #print(f"LinUCB Accuracy: {accuracy_score(optimal_actions_val, lin_UCB_preds):.4f}")
    
    print(f"Average Rewards GPTS: {np.mean(gp_ts_rewards):.4f}, GPTS Accuracy: {accuracy_score(optimal_actions_val, gp_ts_preds):.4f}")
    #print(f"GPTS Accuracy: {accuracy_score(optimal_actions_val, gp_ts_preds):.4f}")
    #print('\n')

In [ ]:
# Example usage
X = df2.values
total_samples = X.shape[0]
n_actions = 2
context_dim = X.shape[1]   # Number of context features
optimal_actions = y

In [ ]:
run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim,  num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=50)
run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim,  num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=50,reward_type='lin2')
run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim,  num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=50,reward_type='lin3')
run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim,  num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=50,reward_type='lin4')
run_backtest_DR(X, total_samples, n_actions, optimal_actions, context_dim,  num_samples=10, lengthscale=1.0, outputscale=1.0, epochs=50,reward_type='lin5')

Average Rewards LinUCB: 1.0415, LinUCB Accuracy: 0.7500
Average Rewards GPTS: 1.0448, GPTS Accuracy: 0.7700
Average Rewards LinUCB: 1.1209, LinUCB Accuracy: 0.6300
Average Rewards GPTS: 1.1314, GPTS Accuracy: 0.6300
Average Rewards LinUCB: 2.0445, LinUCB Accuracy: 0.5200
Average Rewards GPTS: 2.0916, GPTS Accuracy: 0.5500
Average Rewards LinUCB: 1.5758, LinUCB Accuracy: 0.6400
Average Rewards GPTS: 1.6234, GPTS Accuracy: 0.6000
Average Rewards LinUCB: 1.0236, LinUCB Accuracy: 0.6300
Average Rewards GPTS: 1.0389, GPTS Accuracy: 0.6100


## 5. Different alpha for LinUCB

In [ ]:
def run_backtest_with_alpha(X, total_samples, n_actions, optimal_actions, context_dim, reward_type='linear', alpha_values=[0.1, 0.5, 1.0], epochs=20):
    X_train, X_val, historical_actions, historical_rewards, optimal_actions_val, rewards_val = generate_data_and_split(
        X, total_samples, n_actions, optimal_actions, reward_type)

    # Convert PyTorch tensors to NumPy arrays if needed
    if isinstance(X_train, torch.Tensor):
        X_train = X_train.detach().numpy()
    if isinstance(X_val, torch.Tensor):
        X_val = X_val.detach().numpy()
    if isinstance(historical_rewards, torch.Tensor):
        historical_rewards = historical_rewards.detach().numpy()
    if isinstance(historical_actions, torch.Tensor):
        historical_actions = historical_actions.detach().numpy()

    # Generate true rewards based on optimal actions
    all_true_rewards = np.zeros((X_val.shape[0], n_actions))
    reward_gen = RewardGenerator()
    
    for action in range(n_actions):
        for i, opt_action in enumerate(optimal_actions_val):
            if reward_type == 'lin1':
                all_true_rewards[i, action] = reward_gen.generate_rewards1(X_val[i], action, opt_action)
            elif reward_type == 'lin2':
                all_true_rewards[i, action] = reward_gen.generate_rewards2(X_val[i], action, opt_action)
            elif reward_type == 'lin3':
                all_true_rewards[i, action] = reward_gen.generate_rewards3(X_val[i], action, opt_action)
            elif reward_type == 'lin4':
                all_true_rewards[i, action] = reward_gen.generate_rewards4(X_val[i], action, opt_action)
            elif reward_type == 'lin5':
                all_true_rewards[i, action] = reward_gen.generate_rewards5(X_val[i], action, opt_action)

    # Loop over alpha values and calculate DR rewards
    for alpha in alpha_values:
        
        # Initialize LinUCB_DR model with the current alpha
        lin_UCB_DR_model = LinUCB_DR(n_actions, context_dim, alpha=alpha)
        lin_UCB_DR_model.calculate_propensity_scores(historical_actions)  # Calculate propensities internally
        
        # Update the model with historical data
        for i in range(len(historical_actions)):
            lin_UCB_DR_model.update(historical_actions[i], historical_rewards[i, historical_actions[i]], X_train[i])

        # Generate predictions on validation data
        lin_UCB_preds = lin_UCB_DR_model.predict(X_val)
        lin_UCB_preds = np.argmax(lin_UCB_preds, axis=1)
        
        
        # Calculate doubly robust rewards
        dr_rewards = lin_UCB_DR_model.doubly_robust_estimator(X_val, lin_UCB_preds, historical_rewards, historical_actions, all_true_rewards)
        
        # Output average DR rewards for the current alpha
        print(f"Alpha: {alpha}, Average DR Rewards: {np.mean(dr_rewards):.4f}")


In [ ]:
alpha_values = [0.01, 0.1, 1.0, 2.0, 5.0, 10.0, 15.0, 20.0, 50.0]
run_backtest_with_alpha(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type='lin1', alpha_values=alpha_values)

Alpha: 0.01, Average DR Rewards: 1.0497
Alpha: 0.1, Average DR Rewards: 1.0546
Alpha: 1.0, Average DR Rewards: 1.0529
Alpha: 2.0, Average DR Rewards: 1.0371
Alpha: 5.0, Average DR Rewards: 1.0228
Alpha: 10.0, Average DR Rewards: 1.0135
Alpha: 15.0, Average DR Rewards: 1.0086
Alpha: 20.0, Average DR Rewards: 1.0047
Alpha: 50.0, Average DR Rewards: 0.9949


In [ ]:
run_backtest_with_alpha(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type='lin2', alpha_values=alpha_values)

Alpha: 0.01, Average DR Rewards: 0.8731
Alpha: 0.1, Average DR Rewards: 0.8735
Alpha: 1.0, Average DR Rewards: 0.8894
Alpha: 2.0, Average DR Rewards: 0.8816
Alpha: 5.0, Average DR Rewards: 0.8670
Alpha: 10.0, Average DR Rewards: 0.8638
Alpha: 15.0, Average DR Rewards: 0.8504
Alpha: 20.0, Average DR Rewards: 0.8183
Alpha: 50.0, Average DR Rewards: 0.7546


In [ ]:
run_backtest_with_alpha(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type='lin3', alpha_values=alpha_values)

Alpha: 0.01, Average DR Rewards: 2.0868
Alpha: 0.1, Average DR Rewards: 2.0868
Alpha: 1.0, Average DR Rewards: 2.0839
Alpha: 2.0, Average DR Rewards: 2.0866
Alpha: 5.0, Average DR Rewards: 2.0866
Alpha: 10.0, Average DR Rewards: 2.0758
Alpha: 15.0, Average DR Rewards: 2.0971
Alpha: 20.0, Average DR Rewards: 2.0823
Alpha: 50.0, Average DR Rewards: 2.0913


In [ ]:
run_backtest_with_alpha(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type='lin4', alpha_values=alpha_values)

Alpha: 0.01, Average DR Rewards: 1.6129
Alpha: 0.1, Average DR Rewards: 1.6135
Alpha: 1.0, Average DR Rewards: 1.6079
Alpha: 2.0, Average DR Rewards: 1.5888
Alpha: 5.0, Average DR Rewards: 1.5885
Alpha: 10.0, Average DR Rewards: 1.5638
Alpha: 15.0, Average DR Rewards: 1.5766
Alpha: 20.0, Average DR Rewards: 1.5737
Alpha: 50.0, Average DR Rewards: 1.5982


In [ ]:
run_backtest_with_alpha(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type='lin5', alpha_values=alpha_values)

Alpha: 0.01, Average DR Rewards: 1.0490
Alpha: 0.1, Average DR Rewards: 1.0492
Alpha: 1.0, Average DR Rewards: 1.0568
Alpha: 2.0, Average DR Rewards: 1.0605
Alpha: 5.0, Average DR Rewards: 1.0847
Alpha: 10.0, Average DR Rewards: 1.0804
Alpha: 15.0, Average DR Rewards: 1.0638
Alpha: 20.0, Average DR Rewards: 1.0304
Alpha: 50.0, Average DR Rewards: 0.9163


## 6. Statistical Test

In [ ]:
class LinUCB_DR_bias_variance:
    def __init__(self, n_actions, context_dim, alpha=0.5):
        self.n_actions = n_actions
        self.context_dim = context_dim
        self.alpha = alpha
        self.A = [np.eye(context_dim) for _ in range(n_actions)]
        self.b = [np.zeros(context_dim) for _ in range(n_actions)]
        self.propensities = None

    def update(self, action, reward, context):
        """Update A and b matrices for the selected action based on context and observed reward."""
        self.A[action] += np.outer(context, context)
        self.b[action] += reward * context

    def predict(self, X):
        """Predict expected rewards for each action given context X using LinUCB framework."""
        p = np.zeros((X.shape[0], self.n_actions))
        for a in range(self.n_actions):
            theta_a = np.linalg.solve(self.A[a], self.b[a])
            p[:, a] = X @ theta_a + self.alpha * np.sqrt(np.sum(X @ np.linalg.inv(self.A[a]) * X, axis=1))
        return p

    def calculate_propensity_scores(self, historical_actions, imbalance_ratio=None):
        """Calculate propensity scores based on action frequencies, optionally applying imbalance."""
        total_samples = len(historical_actions)
        action_freq = np.bincount(historical_actions, minlength=self.n_actions) / total_samples
        if imbalance_ratio is not None:
            action_freq = np.clip(action_freq * imbalance_ratio, 0, 1)
        self.propensities = action_freq[historical_actions]
        # Debug: print propensities
        # print("Propensities:", self.propensities[:10])  # Print first 10 for a quick check
        return self.propensities

    def doubly_robust_estimator(self, X, chosen_actions, historical_rewards, historical_actions, propensities, all_true_rewards):
        """Compute DR rewards and calculate bias and variance of the estimator."""
        N = len(chosen_actions)
        dr_rewards = np.zeros(N)
        propensities = propensities if propensities is not None else self.propensities

        numerator_all = np.exp(self.predict(X))
        for i in range(N):
            Ai = chosen_actions[i]
            Ri = historical_rewards[i, historical_actions[i]]
            pi_ratio = numerator_all[i, Ai] / np.sum(numerator_all[i])
            predicted_reward = all_true_rewards[i, Ai]
            dr_rewards[i] = pi_ratio * (Ri - predicted_reward) + predicted_reward

        return dr_rewards
    
def calculate_true_policy_value(all_true_rewards):
    """Compute the true policy value as the mean of optimal rewards."""
    optimal_rewards = np.max(all_true_rewards, axis=1)
    return np.mean(optimal_rewards)

def bootstrap_variance(estimator_fn, X, chosen_actions, historical_rewards, historical_actions, propensities, all_true_rewards, n_bootstraps=50):
    """Calculate variance, bias, and MSE using bootstrapping for the given estimator."""
    estimates = []
    true_policy_value = calculate_true_policy_value(all_true_rewards)
    
    for _ in range(n_bootstraps):
        # Resample the data with replacement
        sample_indices = np.random.choice(len(X), len(X), replace=True)
        X_sample = X[sample_indices]
        chosen_actions_sample = chosen_actions[sample_indices]
        historical_rewards_sample = historical_rewards[sample_indices]
        historical_actions_sample = historical_actions[sample_indices]
        propensities_sample = propensities[sample_indices]  # Ensure propensity scores align with resampled data

        # Calculate the estimator's output for the resampled data
        estimate = estimator_fn(X_sample, chosen_actions_sample, historical_rewards_sample, historical_actions_sample, propensities_sample, all_true_rewards)
        estimates.append(np.mean(estimate))

    estimates = np.array(estimates)
    variance = np.var(estimates)
    bias = abs(np.mean(estimates) - true_policy_value)
    mse = bias ** 2 + variance
    
    # # Debug: Print final bias, variance, and mse for validation
    # print("Bias:", bias, "Variance:", variance, "MSE:", mse)
    return variance, bias, mse


In [ ]:
def run_backtest_with_alpha_test(X, total_samples, n_actions, optimal_actions, context_dim, reward_type='linear', alpha_values=[0.1, 0.5, 1.0], epochs=20, imbalance_ratio=None):
    X_train, X_val, historical_actions, historical_rewards, optimal_actions_val, rewards_val = generate_data_and_split(
        X, total_samples, n_actions, optimal_actions, reward_type)
    # Convert PyTorch tensors to NumPy arrays if needed
    if isinstance(X_train, torch.Tensor):
        X_train = X_train.detach().numpy()
    if isinstance(X_val, torch.Tensor):
        X_val = X_val.detach().numpy()
    if isinstance(historical_rewards, torch.Tensor):
        historical_rewards = historical_rewards.detach().numpy()
    if isinstance(historical_actions, torch.Tensor):
        historical_actions = historical_actions.detach().numpy()

    # Generate true rewards based on optimal actions
    all_true_rewards = np.zeros((X_val.shape[0], n_actions))
    reward_gen = RewardGenerator()
    
    for action in range(n_actions):
        for i, opt_action in enumerate(optimal_actions_val):
            if reward_type == 'lin1':
                all_true_rewards[i, action] = reward_gen.generate_rewards1(X_val[i], action, opt_action)
            elif reward_type == 'lin2':
                all_true_rewards[i, action] = reward_gen.generate_rewards2(X_val[i], action, opt_action)
            elif reward_type == 'lin3':
                all_true_rewards[i, action] = reward_gen.generate_rewards3(X_val[i], action, opt_action)
            elif reward_type == 'lin4':
                all_true_rewards[i, action] = reward_gen.generate_rewards4(X_val[i], action, opt_action)
            elif reward_type == 'lin5':
                all_true_rewards[i, action] = reward_gen.generate_rewards5(X_val[i], action, opt_action)
    
    results_df = pd.DataFrame(columns=['Reward Type', 'Alpha', 'Avg DR Rewards', 'Avg LinUCB Rewards', 
                                       'Paired t-stat', 'Paired p-value', 'Observed Mean Diff', 
                                       'Bootstrap p-value', 'Bias DR', 'Bias Non-DR', 'Variance DR', 'Variance Non-DR'])
    
    for alpha in alpha_values:
        lin_UCB_DR_model = LinUCB_DR_bias_variance(n_actions, context_dim, alpha=alpha)
        lin_UCB_DR_model.calculate_propensity_scores(historical_actions, imbalance_ratio=imbalance_ratio)
        
        # Train the LinUCB model
        for i in range(len(historical_actions)):
            lin_UCB_DR_model.update(historical_actions[i], historical_rewards[i, historical_actions[i]], X_train[i])
        
        # Predict actions using LinUCB
        lin_UCB_preds = np.argmax(lin_UCB_DR_model.predict(X_val), axis=1)

        # Calculate DR rewards
        dr_rewards = lin_UCB_DR_model.doubly_robust_estimator(X_val, lin_UCB_preds, historical_rewards, historical_actions, lin_UCB_DR_model.propensities, all_true_rewards)
        dr_rewards_avg = np.mean(dr_rewards)
        
        # Calculate rewards for Non-DR estimator (LinUCB)
        lin_UCB_rewards = np.array([all_true_rewards[i, lin_UCB_preds[i]] for i in range(len(lin_UCB_preds))])
        lin_UCB_rewards_avg = np.mean(lin_UCB_rewards)
        
        # Perform bootstrapping to calculate bias, variance, and MSE for DR and Non-DR estimators
        variance_dr, bias_dr, mse_dr = bootstrap_variance(
            lin_UCB_DR_model.doubly_robust_estimator,
            X_val, lin_UCB_preds, historical_rewards, historical_actions, 
            lin_UCB_DR_model.propensities, all_true_rewards
        )

        # For Non-DR estimator (ensure resampling is consistent)
        variance_non_dr, bias_non_dr, mse_non_dr = bootstrap_variance(
            lambda X, chosen_actions, hr, ha, ps, atr: np.array([atr[i, chosen_actions[i]] for i in range(len(chosen_actions))]),
            X_val, lin_UCB_preds, historical_rewards, historical_actions,
            lin_UCB_DR_model.propensities, all_true_rewards
        )
        
        # Paired t-test between DR and Non-DR rewards
        t_stat, p_value_ttest = stats.ttest_rel(dr_rewards, lin_UCB_rewards)

        # Bootstrap Test
        n_bootstrap = 10000
        differences = dr_rewards - lin_UCB_rewards
        observed_mean_diff = np.mean(differences)
        
        bootstrap_means = np.array([
            np.mean(np.random.choice(differences, size=len(differences), replace=True)) 
            for _ in range(n_bootstrap)
        ])
        
        p_value_bootstrap = np.mean(bootstrap_means >= observed_mean_diff)
        
        # Append results to the DataFrame
        temp_df = pd.DataFrame([{
            'Reward Type': reward_type,
            'Alpha': alpha,
            'Avg DR Rewards': dr_rewards_avg,
            'Avg LinUCB Rewards': lin_UCB_rewards_avg,
            'Paired t-stat': t_stat,
            'Paired p-value': p_value_ttest,
            'Observed Mean Diff': observed_mean_diff,
            'Bootstrap p-value': p_value_bootstrap,
            'Bias DR': bias_dr,
            'Bias Non-DR': bias_non_dr,
            'Variance DR': variance_dr,
            'Variance Non-DR': variance_non_dr
        }])

        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    return results_df

In [ ]:
# Running the backtest
alpha_values = [0.01, 0.1, 1.0, 10.0, 50.0]
imbalance_ratio = 1.5  # Test different imbalance levels as desired
results_all = pd.DataFrame()

for reward_type in ['lin1', 'lin2', 'lin3', 'lin4', 'lin5']:
    result_df = run_backtest_with_alpha_test(
        X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, 
        context_dim=X.shape[1], reward_type=reward_type, 
        alpha_values=alpha_values, imbalance_ratio=imbalance_ratio)
    results_all = pd.concat([results_all, result_df], ignore_index=True)

# Display or save final results DataFrame
results_all

,Reward Type,Alpha,Avg DR Rewards,Avg LinUCB Rewards,Paired t-stat,Paired p-value,Observed Mean Diff,Bootstrap p-value
0,lin1,0.01,1.036236,1.219565,-6.047755,2.615536e-08,-0.183329,0.4973
1,lin1,0.10,1.036087,1.219565,-6.047007,2.624411e-08,-0.183478,0.4940
2,lin1,1.00,1.027776,1.205657,-5.843505,6.552924e-08,-0.177880,0.4982
3,lin1,2.00,1.012747,1.179694,-5.259199,8.367818e-07,-0.166946,0.5021
4,lin1,5.00,0.985659,1.142327,-4.473606,2.055800e-05,-0.156668,0.5094
5,lin1,10.00,0.968308,1.133278,-4.251347,4.817892e-05,-0.164970,0.4879
6,lin1,15.00,0.954484,1.126560,-4.068652,9.512833e-05,-0.172076,0.4929
7,lin1,20.00,0.946410,1.126560,-4.008388,1.185785e-04,-0.180150,0.4967
8,lin1,50.00,0.926750,1.126560,-3.849318,2.100248e-04,-0.199809,0.4937
9,lin2,0.01,1.121617,1.191350,-0.729376,4.674937e-01,-0.069734,0.4931


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

def run_backtest_with_alpha_test_with_exploration(X, total_samples, n_actions, optimal_actions, context_dim, reward_type='linear', alpha_values=[0.1, 0.5, 1.0], epochs=20):
    X_train, X_val, historical_actions, historical_rewards, optimal_actions_val, rewards_val = generate_data_and_split(
        X, total_samples, n_actions, optimal_actions, reward_type)

    # Convert PyTorch tensors to NumPy arrays if needed
    if isinstance(X_train, torch.Tensor):
        X_train = X_train.detach().numpy()
    if isinstance(X_val, torch.Tensor):
        X_val = X_val.detach().numpy()
    if isinstance(historical_rewards, torch.Tensor):
        historical_rewards = historical_rewards.detach().numpy()
    if isinstance(historical_actions, torch.Tensor):
        historical_actions = historical_actions.detach().numpy()

    # Generate true rewards based on optimal actions
    all_true_rewards = np.zeros((X_val.shape[0], n_actions))
    reward_gen = RewardGenerator()
    
    for action in range(n_actions):
        for i, opt_action in enumerate(optimal_actions_val):
            if reward_type == 'lin1':
                all_true_rewards[i, action] = reward_gen.generate_rewards1(X_val[i], action, opt_action)
            elif reward_type == 'lin2':
                all_true_rewards[i, action] = reward_gen.generate_rewards2(X_val[i], action, opt_action)
            elif reward_type == 'lin3':
                all_true_rewards[i, action] = reward_gen.generate_rewards3(X_val[i], action, opt_action)
            elif reward_type == 'lin4':
                all_true_rewards[i, action] = reward_gen.generate_rewards4(X_val[i], action, opt_action)
            elif reward_type == 'lin5':
                all_true_rewards[i, action] = reward_gen.generate_rewards5(X_val[i], action, opt_action)

    # Create an empty DataFrame to store results
    results_df = pd.DataFrame(columns=['Reward Type', 'Alpha Pair', 'Avg DR Rewards (No Exploration)', 'Avg DR Rewards (With Exploration)', 'Paired t-stat', 'Paired p-value', 'Observed Mean Diff', 'Bootstrap p-value'])

    # Store DR rewards for each alpha to compare later
    dr_rewards_dict = {}

    for alpha in alpha_values:
        # Initialize LinUCB model with current alpha
        lin_UCB_DR_model = LinUCB_DR(n_actions, context_dim, alpha=alpha)
        lin_UCB_DR_model.calculate_propensity_scores(historical_actions)  # Calculate propensities internally
        
        # Train LinUCB model on historical data
        for i in range(len(historical_actions)):
            lin_UCB_DR_model.update(historical_actions[i], historical_rewards[i, historical_actions[i]], X_train[i])
        
        # Predict with LinUCB on validation set
        lin_UCB_preds = lin_UCB_DR_model.predict(X_val)
        lin_UCB_preds = np.argmax(lin_UCB_preds, axis=1)
        
        # Calculate doubly robust rewards
        dr_rewards = lin_UCB_DR_model.doubly_robust_estimator(X_val, lin_UCB_preds, historical_rewards, historical_actions, all_true_rewards)
        dr_rewards_avg = np.mean(dr_rewards)
        
        # Store the DR rewards for the current alpha value
        dr_rewards_dict[alpha] = dr_rewards

    # Perform paired t-test and bootstrap test between alpha=0.01 (without exploration) and other alphas
    alpha_no_exploration = 0.01
    dr_rewards_no_exploration = dr_rewards_dict[alpha_no_exploration]

    for alpha_with_exploration in alpha_values:
        if alpha_with_exploration == alpha_no_exploration:
            continue  # Skip comparison of 0.01 with itself

        dr_rewards_with_exploration = dr_rewards_dict[alpha_with_exploration]

        # Paired t-test between DR rewards with and without exploration
        t_stat, p_value_ttest = stats.ttest_rel(dr_rewards_no_exploration, dr_rewards_with_exploration)

        # Bootstrap Test
        n_bootstrap = 10000
        differences = dr_rewards_with_exploration - dr_rewards_no_exploration
        observed_mean_diff = np.mean(differences)
        
        # Bootstrap sampling
        bootstrap_means = np.array([
            np.mean(np.random.choice(differences, size=len(differences), replace=True)) 
            for _ in range(n_bootstrap)
        ])
        
        # Calculate the p-value as the proportion of bootstrap samples with a mean greater than or equal to the observed mean difference
        p_value_bootstrap = np.mean(bootstrap_means >= observed_mean_diff)
        
        # Append comparison results to the DataFrame
        temp_df = pd.DataFrame([{
            'Reward Type': reward_type,
            'Alpha Pair': f"{alpha_no_exploration} vs {alpha_with_exploration}",
            'Avg DR Rewards (No Exploration)': np.mean(dr_rewards_no_exploration),
            'Avg DR Rewards (With Exploration)': np.mean(dr_rewards_with_exploration),
            'Paired t-stat': t_stat,
            'Paired p-value': p_value_ttest,
            'Observed Mean Diff': observed_mean_diff,
            'Bootstrap p-value': p_value_bootstrap
        }])
        
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    return results_df

# Running the function with a loop over different reward types and alpha values
alpha_values = [0.01, 5.0, 20.0, 50.0]
results_all = pd.DataFrame()
for reward_type in ['lin1', 'lin2', 'lin3', 'lin4', 'lin5']:
    result_df = run_backtest_with_alpha_test_with_exploration(X, total_samples=1000, n_actions=2, optimal_actions=optimal_actions, context_dim=X.shape[1], reward_type=reward_type, alpha_values=alpha_values)
    results_all = pd.concat([results_all, result_df], ignore_index=True)

# Display the final results DataFrame
display(results_all)


,Reward Type,Alpha Pair,Avg DR Rewards (No Exploration),Avg DR Rewards (With Exploration),Paired t-stat,Paired p-value,Observed Mean Diff,Bootstrap p-value
0,lin1,0.01 vs 5.0,1.002683,0.987286,0.971970,0.333434,-0.015397,0.4920
1,lin1,0.01 vs 20.0,1.002683,0.946410,2.827290,0.005681,-0.056273,0.4927
2,lin1,0.01 vs 50.0,1.002683,0.921676,3.445175,0.000839,-0.081006,0.4955
3,lin2,0.01 vs 5.0,1.165367,1.129588,1.498521,0.137180,-0.035779,0.5203
4,lin2,0.01 vs 20.0,1.165367,1.113143,1.575056,0.118434,-0.052224,0.5051
5,lin2,0.01 vs 50.0,1.165367,1.119896,1.049447,0.296528,-0.045471,0.5076
6,lin3,0.01 vs 5.0,2.015294,1.990365,1.161197,0.248354,-0.024929,0.5223
7,lin3,0.01 vs 20.0,2.015294,1.972605,1.747295,0.083687,-0.042690,0.4974
8,lin3,0.01 vs 50.0,2.015294,2.002913,0.555503,0.579803,-0.012381,0.5094
9,lin4,0.01 vs 5.0,1.530217,1.523874,0.441207,0.660025,-0.006343,0.5327
